# Imports

In [4]:
# %%capture         # silent cell
import numpy as np
import csv
from fastlmm.association import gsm_extraction
from fastlmm.inference.fastlmm_predictor import _snps_fixup, _pheno_fixup, _kernel_fixup, _SnpTrainTest
from fastlmm.inference.lmm_cov import LMM

C:\Users\micha\Anaconda2\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
C:\Users\micha\Anaconda2\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


# Data

The `dgrp2.bed/bim/fam` file triad contains all nuclear variants of freeze 2 DGRP that are described in the Huang2014 paper. These files can be downloaded here: http://dgrp2.gnets.ncsu.edu/data.html.

You can put any set of variants you want, specific chromosomes, etc.

In [5]:
OUTPUT_LOCATION = '../Outputs/'

# OUTPUT_NAME     = 'GSM_DGRPFreeze2'
OUTPUT_NAME     = 'GSM_Mito_Full'

# VARIANTS = '../Raw_Data/dgrp2'
VARIANTS = '../Inputs/MitoSeq_AllRuns_dm6_chrM.annot.biallellic_ConvertedReference'

# GSM Extraction

The following code snippet generates a random phenotype because phenotype values are not needed for GSM calculation (the the `gsm_extraction` function just needs a phenotype input anyway).

In [6]:
test_snps = _snps_fixup(VARIANTS, count_A1=None)

test_snps.iid.tofile('../Inputs/GSM_Extraction_Pheno.txt', sep='\n')

my_list = []
with open('../Inputs/GSM_Extraction_Pheno.txt', 'r') as f:
    for count, line in enumerate(f, start=1):
        if count % 2 == 0:
            tmp = line.replace("\'", "")
            tmp = tmp.replace("\n", "")
            my_list.append(tmp)
            
with open('../Inputs/GSM_Extraction_Pheno.txt', 'w') as f:
    for line in my_list:
        f.write(' '.join([line, line, str(0.333)+"\n"]))


C:\Users\micha\Anaconda2\lib\site-packages\pysnptools-0.3.13-py2.7-win-amd64.egg\pysnptools\snpreader\bed.py:42: FutureWarning: 'count_A1' was not set. For now it will default to 'False', but in the future it will default to 'True'
  warnings.warn("'count_A1' was not set. For now it will default to 'False', but in the future it will default to 'True'", FutureWarning)


I slightly modified the FaST-LMM function `single_snp` so that it only does the GSM calculation part without succeeding GWAS. You can choose to calculate the GSM with the "leave out one chrom" method if you want; the default is `False`. (The results between False and True are very similar for all *D. melanogaster* data I tested.)

In [ ]:
gsm_extraction(VARIANTS,
               '../Inputs/GSM_Extraction_Pheno.txt',
               cache_file = OUTPUT_LOCATION+OUTPUT_NAME,
               leave_out_one_chrom = False)

# Everything below can be ignored

In [ ]:
#gsm_extraction_leave_out_one_chrom_false(variants_nuclear,
#           tergite5_pheno,
#           cache_file = '../Outputs/Fast-Lmm-Outputs/GsmNuclear-Tergite.txt')


#lmm_nuclear = LMM()
#with np.load('../Outputs/Fast-Lmm-Cache/Cache-Nuclear.npz') as data:
#    nuclear_U = data['arr_0']
#    nuclear_S_vector = data['arr_1']

#nuclear_S = np.diag(nuclear_S_vector)
#
#nuclear_K = np.dot(np.dot(nuclear_U, nuclear_S), nuclear_U)
#            
#nmm_nuclear.get_K


# To run manually
# gsm_extraction_leave_out_one_chrom_false(variants_mito,
#            random_artificial_pheno_mito,
#            cache_file = '../Outputs/Fast-Lmm-Outputs/GsmMito-171DgrpLines.txt')
# gsm_extraction_leave_out_one_chrom_false(variants_mito,
#            random_artificial_pheno_mito_with_non_dgrp,
#                cache_file = '../Outputs/Fast-Lmm-Outputs/GsmMito-All179Lines.txt')

### Spectral Decomposition of NCSU GSM

In [96]:
from pysnptools.kernelreader import KernelData

import re
import sys

with open('../Raw_Data/freeze2.common.rel.mat') as f:
    first_line = f.readline()
    
# print first_line

tmp = re.split(r'\t+', first_line.rstrip())
tmp = tmp[1:]
# print tmp
iids = []
for i in range(len(tmp)):
     iids.append(re.split(r' +', tmp[i]))

        
Gsm = np.genfromtxt('../Raw_Data/freeze2.common.rel.mat', skip_header=True)
Gsm = Gsm[:, 2:207]
# Gsm = LMM(K=Gsm)
# Gsm.setSU_fromK()
# Gsm.K

# np.savetxt('../Inputs/NCSU_GSM_U.txt', Gsm.U)
# np.savetxt('../Inputs/NCSU_GSM_S.txt', Gsm.S)


my_kernel = KernelData(iid=iids, val=Gsm)

[['line_21' 'line_21']
 ['line_26' 'line_26']
 ['line_28' 'line_28']
 ['line_31' 'line_31']
 ['line_32' 'line_32']
 ['line_38' 'line_38']
 ['line_40' 'line_40']
 ['line_41' 'line_41']
 ['line_42' 'line_42']
 ['line_45' 'line_45']
 ['line_48' 'line_48']
 ['line_49' 'line_49']
 ['line_57' 'line_57']
 ['line_59' 'line_59']
 ['line_69' 'line_69']
 ['line_73' 'line_73']
 ['line_75' 'line_75']
 ['line_83' 'line_83']
 ['line_85' 'line_85']
 ['line_88' 'line_88']
 ['line_91' 'line_91']
 ['line_93' 'line_93']
 ['line_100' 'line_100']
 ['line_101' 'line_101']
 ['line_105' 'line_105']
 ['line_109' 'line_109']
 ['line_129' 'line_129']
 ['line_136' 'line_136']
 ['line_138' 'line_138']
 ['line_142' 'line_142']
 ['line_149' 'line_149']
 ['line_153' 'line_153']
 ['line_158' 'line_158']
 ['line_161' 'line_161']
 ['line_176' 'line_176']
 ['line_177' 'line_177']
 ['line_181' 'line_181']
 ['line_189' 'line_189']
 ['line_195' 'line_195']
 ['line_208' 'line_208']
 ['line_217' 'line_217']
 ['line_223' 'line_

In [66]:
tmp = 'fdfaj;lsf \n'
print tmp.rstrip()

fdfaj;lsf


In [14]:
# Gsm.setK()
# Gsm.getK()
Gsm.findH2()

AttributeError: 'NoneType' object has no attribute 'shape'